In [ ]:
from tensorflow import keras
import tensorflow_datasets as tfds
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
import tensorflow as tf
import numpy as np
import glob
from shutil import copyfile
from PIL import Image

In [ ]:
!git clone https://github.com/ultralytics/yolov3
%cd yolov3
!pip install -r requirements.txt

import pathlib
import os
import cv2
import zipfile
from os import path
import shutil
import csv
import random
import numpy as np

#Refer to Notebooks/Yolov5/SeniorDesignYolov5.ipynb to compress the files and then upload the .zip file.

In [ ]:
!unzip ./LPCVtrain.zip

In [ ]:
%mkdir ./LPCV/valid
%mkdir ./LPCV/valid/images
%mkdir ./LPCV/valid/labels

In [ ]:
def contrast_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_contrast(img, 2, 5, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "contrast", folder)
def noise_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.stateless_random_jpeg_quality(img, 2, 20, (42,42))
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "noise", folder)
def grayscale_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rgb_to_grayscale(img)
    img = img.numpy()
    img = img.reshape((img.shape[0], img.shape[1]))
    im = Image.fromarray(img, 'L')
    im.save(folder+'/images/'+new_image)
    copy_label(image, "grayscale", folder)
def saturate_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_saturation(img, 2, 120, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "saturate", folder)
def hue_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_hue(img, 0.1)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "hue", folder)
def flip_image_lr(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_left_right(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_lr(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_lr.txt", folder)
def flip_image_ud(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_up_down(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_ud(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_ud.txt", folder)
def rot_90(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rot90(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    label_rot_90(image[:image.find('.')]+".txt", image[:image.find('.')]+"_rot_90.txt", folder)
def flip_label_lr(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[1] = str(1-float(words[1]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def flip_label_ud(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[2] = str(1-float(words[2]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def label_rot_90(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            x = words[1]
            y = words[2]
            w = words[3]
            h = words[4]
            words[1] = y
            words[2] = str(1-float(x))
            words[3] = h
            words[4] = w
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def copy_label(image, augmentation, folder):
    filename = image[:image.find('.')]
    new_filename = folder+'/labels/'+filename+'_'+augmentation+'.txt'
    filename = folder+'/labels/'+filename+'.txt'
    copyfile(filename, new_filename)

In [ ]:
aug_list = ['_contrast', '_noise', '_flip_lr', '_flip_ud', '_rot_90', '_saturate', '_hue']
data_dir = "./LPCV/train"

In [ ]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
    for file in files:
      count = 0
      for j in aug_list:
        filename = file[:file.find('.')]+j+file[file.find('.'):]
        if count==0: contrast_image(file, filename, data_dir)
        if count==1: noise_image(file, filename, data_dir)
        if count==2: flip_image_lr(file, filename, data_dir)
        if count==3: flip_image_ud(file, filename, data_dir)
        if count==4: rot_90(file, filename, data_dir)
        if count==5: saturate_image(file, filename, data_dir)
        if count==6: hue_image(file, filename, data_dir)
        count+=1

In [ ]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
    for file in files:
        filename = file[:file.find('.')]+"_grayscale"+file[file.find('.'):]
        grayscale_image(file, filename, data_dir)

In [ ]:
def moveFiles(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=12).permutation(files)[:300]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [ ]:
moveFiles("./LPCV/train/images", "./LPCV/valid/images")
moveFiles("./LPCV/train/labels", "./LPCV/valid/labels")

In [ ]:
data = open("data.yaml", "w")
data.write("train: /content/yolov3/content/yolov3/LPCV/train/images\nval: /content/yolov3/content/yolov3/LPCV/valid/images\nnc: 2\nnames: [\'person\', \'sports-ball\']")
data.close()

In [ ]:
!python train.py --img 416 --batch 150 --epochs 20 --data data.yaml --weights yolov3-tiny.pt

In [ ]:
!python detect.py --weights ./runs/train/exp/weights/best.pt --img 256 --conf 0.25 --source 5p5b_03A1.mp4 # uplaod one of the videos

In [ ]:
!python /content/yolov3/val.py --weights /content/yolov3/runs/train/exp/weights/best.pt --img 640 --data /content/yolov3/data.yaml

In [ ]:
!python export.py --weights ./runs/train/exp/weights/best.pt --include tflite --img-size 256


# not working as Dec/21